# Model Building

In [1]:
#Import the required Libraries
import numpy as np
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
raw_data = pd.read_csv("Trady Flow - Best Options Trade Ideas.csv")
pd.options.display.max_columns = None
raw_data.head()

,Time,Sym,C/P,Exp,Strike,Spot,BidAsk,Orders,Vol,Prems,OI,Diff(%),ITM
0,6/17/2022 15:07,ISEE,Call,10/21/2022,10.0,9.54,5.05,7,360,183.60K,4.07K,4.71,0
1,6/17/2022 15:05,CVNA,Call,1/19/2024,60.0,23.52,4.60,7,634,310.66K,130,155.05,0
2,6/17/2022 14:51,PTLO,Put,2/17/2023,15.0,15.19,3.50,7,800,281.00K,0,1.39,0
3,6/17/2022 14:39,TWLO,Call,6/24/2022,86.0,84.51,2.95,5,722,198.80K,436,2.48,0
4,6/17/2022 13:56,ATUS,Put,9/16/2022,7.0,8.62,0.68,5,6.27K,501.84K,8.63K,23.13,0


In [3]:
raw_data['Date'] = pd.to_datetime(raw_data['Time']).dt.date

In [4]:
raw_data.head()

,Time,Sym,C/P,Exp,Strike,Spot,BidAsk,Orders,Vol,Prems,OI,Diff(%),ITM,Date
0,6/17/2022 15:07,ISEE,Call,10/21/2022,10.0,9.54,5.05,7,360,183.60K,4.07K,4.71,0,2022-06-17
1,6/17/2022 15:05,CVNA,Call,1/19/2024,60.0,23.52,4.60,7,634,310.66K,130,155.05,0,2022-06-17
2,6/17/2022 14:51,PTLO,Put,2/17/2023,15.0,15.19,3.50,7,800,281.00K,0,1.39,0,2022-06-17
3,6/17/2022 14:39,TWLO,Call,6/24/2022,86.0,84.51,2.95,5,722,198.80K,436,2.48,0,2022-06-17
4,6/17/2022 13:56,ATUS,Put,9/16/2022,7.0,8.62,0.68,5,6.27K,501.84K,8.63K,23.13,0,2022-06-17


In [5]:
raw_data.shape

(7827, 14)

In [6]:
raw_data.groupby('Sym').size()

Sym
A         2
AA       41
AAL      57
AAPL     48
ABBV     19
ABC       1
ABCL      1
ABNB     16
ABT       4
ABUS      1
ACAD      1
ACB       1
ACI       1
ACM       1
ACMR      6
ACN       4
ACWI      1
ADBE      8
ADI      19
ADM       1
ADNT      1
ADP       1
ADS       4
ADSK      2
ADT       1
AEHR      1
AEM       4
AEO      11
AER       2
AES       4
AFRM     47
AG        3
AGC       3
AHT       2
AI        5
AIG       2
AKAM     18
ALB       5
ALEC      1
ALGN      1
ALGT      4
ALK       3
ALL       1
ALLY      3
ALNY      2
AMAT     17
AMBA      2
AMC      30
AMCX      1
AMD      50
AMED      1
AMGN     10
AMRS      1
AMT       2
AMZN     30
AN        2
ANF       5
ANTM      2
ANY       3
AON       4
AOUT      3
APA       8
APLS      1
APO       1
APP       1
APPS      6
APSG      1
AR       18
ARCT      1
ARE       1
ARKG      3
ARKK     23
ARNC      5
ARQQ      2
ARRY      3
ARVL      4
ARVN      2
ARWR      1
ASAN      5
ASHR      1
ASLE      1
ASML      2
ASO      20


#### Renaming columns

In [7]:
raw_data.rename(columns={raw_data.columns[1]: 'Symbols', raw_data.columns[2]: 'Option_type', raw_data.columns[3]: 'Expiry', raw_data.columns[4]: 'Strike_price', raw_data.columns[5]: 'Spot_price', raw_data.columns[8]: 'Volume', raw_data.columns[9]: 'Premium'},inplace=True)
raw_data.head()

,Time,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,OI,Diff(%),ITM,Date
0,6/17/2022 15:07,ISEE,Call,10/21/2022,10.0,9.54,5.05,7,360,183.60K,4.07K,4.71,0,2022-06-17
1,6/17/2022 15:05,CVNA,Call,1/19/2024,60.0,23.52,4.60,7,634,310.66K,130,155.05,0,2022-06-17
2,6/17/2022 14:51,PTLO,Put,2/17/2023,15.0,15.19,3.50,7,800,281.00K,0,1.39,0,2022-06-17
3,6/17/2022 14:39,TWLO,Call,6/24/2022,86.0,84.51,2.95,5,722,198.80K,436,2.48,0,2022-06-17
4,6/17/2022 13:56,ATUS,Put,9/16/2022,7.0,8.62,0.68,5,6.27K,501.84K,8.63K,23.13,0,2022-06-17


#### Handling Datatypes

In [8]:
raw_data['Expiry'] = pd.to_datetime(raw_data['Expiry']).dt.date
raw_data.head()

,Time,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,OI,Diff(%),ITM,Date
0,6/17/2022 15:07,ISEE,Call,2022-10-21,10.0,9.54,5.05,7,360,183.60K,4.07K,4.71,0,2022-06-17
1,6/17/2022 15:05,CVNA,Call,2024-01-19,60.0,23.52,4.60,7,634,310.66K,130,155.05,0,2022-06-17
2,6/17/2022 14:51,PTLO,Put,2023-02-17,15.0,15.19,3.50,7,800,281.00K,0,1.39,0,2022-06-17
3,6/17/2022 14:39,TWLO,Call,2022-06-24,86.0,84.51,2.95,5,722,198.80K,436,2.48,0,2022-06-17
4,6/17/2022 13:56,ATUS,Put,2022-09-16,7.0,8.62,0.68,5,6.27K,501.84K,8.63K,23.13,0,2022-06-17


In [9]:
raw_data.dtypes

Time             object
Symbols          object
Option_type      object
Expiry           object
Strike_price    float64
Spot_price      float64
BidAsk          float64
Orders            int64
Volume           object
Premium          object
OI               object
Diff(%)         float64
ITM               int64
Date             object
dtype: object

In [10]:
repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9', }
raw_data['Volume'] = raw_data['Volume'].replace(repl_dict, regex=True).map(pd.eval)
raw_data['Premium'] = raw_data['Premium'].replace(repl_dict, regex=True).map(pd.eval)

In [11]:
raw_data.dtypes

Time             object
Symbols          object
Option_type      object
Expiry           object
Strike_price    float64
Spot_price      float64
BidAsk          float64
Orders            int64
Volume          float64
Premium         float64
OI               object
Diff(%)         float64
ITM               int64
Date             object
dtype: object

In [12]:
raw_data.describe()

,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),ITM
count,7827.000000,7827.000000,7827.000000,7827.000000,7827.000000,7.827000e+03,7827.000000,7827.000000
mean,151.178342,148.595490,4.220649,7.109493,2740.656573,7.189672e+05,11.477625,0.679826
std,358.668235,353.566766,5.207856,5.312003,4931.768058,1.484765e+06,19.367772,0.466573
min,1.500000,1.230000,0.110000,5.000000,250.000000,1.252500e+05,0.020000,0.000000
25%,30.000000,28.270000,1.650000,5.000000,708.000000,2.388650e+05,2.110000,0.000000
50%,60.000000,58.950000,3.100000,5.000000,1310.000000,3.676300e+05,5.600000,1.000000
75%,145.000000,145.160000,5.120000,7.000000,2810.000000,6.749750e+05,12.715000,1.000000
max,3950.000000,3755.250000,194.350000,124.000000,131530.000000,6.235000e+07,277.640000,1.000000


#### Dropping insignificant columns from the data

In [13]:
raw_data = raw_data.drop(['Time','OI','ITM'], 1)
raw_data.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date
0,ISEE,Call,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17
1,CVNA,Call,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17
2,PTLO,Put,2023-02-17,15.0,15.19,3.50,7,800.0,281000.0,1.39,2022-06-17
3,TWLO,Call,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17
4,ATUS,Put,2022-09-16,7.0,8.62,0.68,5,6270.0,501840.0,23.13,2022-06-17


In [14]:
raw_data.describe()

,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%)
count,7827.000000,7827.000000,7827.000000,7827.000000,7827.000000,7.827000e+03,7827.000000
mean,151.178342,148.595490,4.220649,7.109493,2740.656573,7.189672e+05,11.477625
std,358.668235,353.566766,5.207856,5.312003,4931.768058,1.484765e+06,19.367772
min,1.500000,1.230000,0.110000,5.000000,250.000000,1.252500e+05,0.020000
25%,30.000000,28.270000,1.650000,5.000000,708.000000,2.388650e+05,2.110000
50%,60.000000,58.950000,3.100000,5.000000,1310.000000,3.676300e+05,5.600000
75%,145.000000,145.160000,5.120000,7.000000,2810.000000,6.749750e+05,12.715000
max,3950.000000,3755.250000,194.350000,124.000000,131530.000000,6.235000e+07,277.640000


In [15]:
raw_data.shape

(7827, 11)

#### Filter the dataset only for "Call"

In [16]:
df_call = raw_data.loc[(raw_data['Option_type'] == 'Call')]
df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date
0,ISEE,Call,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17
1,CVNA,Call,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17
3,TWLO,Call,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17
8,ET,Call,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17
9,WFC,Call,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17


In [17]:
# Replacing 'Call' value with 'c' for Implied Volatility calculation
df_call['Option_type'] = df_call['Option_type'].replace('Call','c')

In [18]:
df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17


In [19]:
df_call.dtypes

Symbols          object
Option_type      object
Expiry           object
Strike_price    float64
Spot_price      float64
BidAsk          float64
Orders            int64
Volume          float64
Premium         float64
Diff(%)         float64
Date             object
dtype: object

In [20]:
df_call.isnull().sum()

Symbols         0
Option_type     0
Expiry          0
Strike_price    0
Spot_price      0
BidAsk          0
Orders          0
Volume          0
Premium         0
Diff(%)         0
Date            0
dtype: int64

#### Calculating 'Time_to_Expiry' col
Time_to_Expiry = Expiry - Date / 250 (Market functioning days)

In [21]:
df_call['Days_diff'] = (df_call['Expiry'] - df_call['Date']).dt.days
df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581


In [22]:
df_call['Time_to_Expiry'] = (df_call['Days_diff']/250)
df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff,Time_to_Expiry
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126,0.504
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581,2.324
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7,0.028
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581,2.324
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581,2.324


#### Compute volatility using Pandas rolling and std methods, the trading days is set to 365 days

TRADING_DAYS = 365
returns = np.log(raw_data['Spot']/raw_data['Spot'].shift(1))
returns.fillna(0, inplace=True)
volatility = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
volatility.tail()

%matplotlib inline
fig = plt.figure(figsize=(15, 7))
ax1 = fig.add_subplot(1, 1, 1)
volatility.plot(ax=ax1)
ax1.set_xlabel('Date')
ax1.set_ylabel('Volatility')
ax1.set_title('Annualized volatility for input-Raw data')
plt.show()

TRADING_DAYS = 365
returns = np.log(raw_data['Spot']/raw_data['Spot'].shift(1))
returns.fillna(0, inplace=True)
raw_data["volatility"] = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
raw_data.tail()

raw_data.shape

raw_data.describe()

raw_data["volatility"].isnull().sum()

raw_data["volatility"].isnull().sum()/len(raw_data) * 100

### Calculationg Implied Volatility of Stock Option

In [23]:
df_call.dtypes

Symbols            object
Option_type        object
Expiry             object
Strike_price      float64
Spot_price        float64
BidAsk            float64
Orders              int64
Volume            float64
Premium           float64
Diff(%)           float64
Date               object
Days_diff           int64
Time_to_Expiry    float64
dtype: object

In [24]:
df_call['BidAsk']

0       5.05
1       4.60
3       2.95
8       1.53
9       6.15
        ... 
7819    5.50
7820    0.89
7822    4.62
7823    1.38
7826    5.40
Name: BidAsk, Length: 5077, dtype: float64

price = df_call['BidAsk']
s = df_call['Spot_price']
k = df_call['Strike_price']
t = df_call['Time_to_Expiry']
r = 0
q = 0
flag = 'c'

price = 5.05
s = 9.54
k = 10.0
t = 0.504
r = 0
q = 0
flag = 'c'

iv = implied_volatility(price, s, k, t, r, q, flag)
print(iv)

- <b>price</b>: The option's market price.
- s: The current price of the stock.
- k: The option's strike price.
- t: The time until expiration, expressed in years.
- r: The risk-free interest rate, expressed as a decimal.
- q: annualized continuous dividend rate.
- flag: A string indicating whether the option is a call option ('c') or a put option ('p').

https://stackoverflow.com/questions/61289020/fast-implied-volatility-calculation-in-python

import numpy as np
from scipy.stats import norm
N = norm.cdf

def bs_call(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def find_vol(target_value, S, K, T, r, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_call(S, K, T, r, sigma)
        vega = bs_vega(S, K, T, r, sigma)
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

In [25]:
df_call.describe()

,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Days_diff,Time_to_Expiry
count,5077.000000,5077.000000,5077.000000,5077.000000,5077.000000,5.077000e+03,5077.000000,5077.000000,5077.000000
mean,164.266939,155.124824,4.056494,7.295647,2801.577112,7.029790e+05,11.661036,113.250148,0.453001
std,392.983901,380.989119,4.429347,6.049169,4983.100369,1.475597e+06,21.035417,147.217554,0.588870
min,1.500000,1.230000,0.110000,5.000000,250.000000,1.252500e+05,0.020000,0.000000,0.000000
25%,30.000000,27.020000,1.520000,5.000000,756.000000,2.364600e+05,1.940000,21.000000,0.084000
50%,62.500000,57.350000,2.930000,5.000000,1390.000000,3.639100e+05,5.200000,53.000000,0.212000
75%,152.500000,147.080000,5.050000,7.000000,2940.000000,6.661800e+05,12.060000,150.000000,0.600000
max,3950.000000,3755.250000,72.900000,124.000000,131530.000000,6.235000e+07,267.290000,850.000000,3.400000


S = df_call['Strike_price']
K = df_call['Spot_price']
T = df_call['Time_to_Expiry']
r = 0
vol = df_call['Volume']

V_market = bs_call(S, K, T, r, vol)
implied_vol = find_vol(V_market, S, K, T, r)

print ('Implied vol: %.2f%%' % (implied_vol * 100))
print ('Market price = %.2f' % V_market)
print ('Model price = %.2f' % bs_call(S, K, T, r, implied_vol))

In [26]:
import py_vollib.black_scholes.implied_volatility as iv

# Define a function to calculate implied volatility
def calculate_implied_volatility(row):
    S = row['Spot_price']
    K = row['Strike_price']
    r = 0  # Risk-free interest rate (set to 0 for simplicity)
    t = row['Time_to_Expiry']
    price = row['BidAsk']
    option_type = row['Option_type']
    
    try:
        implied_volatility = iv.implied_volatility(price, S, K, t, r, option_type)
    except:
        implied_volatility = None
    
    return implied_volatility

# Apply the function to calculate implied volatility for each row in the DataFrame
df_call['Implied_Volatility'] = df_call.apply(calculate_implied_volatility, axis=1)

In [27]:
df_call['Implied_Volatility'].value_counts

<bound method IndexOpsMixin.value_counts of 0       2.082404
1       0.754821
3       0.641070
8       0.268521
9       0.294886
          ...   
7819    0.719506
7820    1.054175
7822    0.228837
7823    0.610646
7826    0.392591
Name: Implied_Volatility, Length: 5077, dtype: float64>

In [28]:
df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff,Time_to_Expiry,Implied_Volatility
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126,0.504,2.082404
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581,2.324,0.754821
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7,0.028,0.641070
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581,2.324,0.268521
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581,2.324,0.294886


In [29]:
df_call.describe()

,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Days_diff,Time_to_Expiry,Implied_Volatility
count,5077.000000,5077.000000,5077.000000,5077.000000,5077.000000,5.077000e+03,5077.000000,5077.000000,5077.000000,5076.000000
mean,164.266939,155.124824,4.056494,7.295647,2801.577112,7.029790e+05,11.661036,113.250148,0.453001,inf
std,392.983901,380.989119,4.429347,6.049169,4983.100369,1.475597e+06,21.035417,147.217554,0.588870,NaN
min,1.500000,1.230000,0.110000,5.000000,250.000000,1.252500e+05,0.020000,0.000000,0.000000,0.071907
25%,30.000000,27.020000,1.520000,5.000000,756.000000,2.364600e+05,1.940000,21.000000,0.084000,0.269261
50%,62.500000,57.350000,2.930000,5.000000,1390.000000,3.639100e+05,5.200000,53.000000,0.212000,0.396793
75%,152.500000,147.080000,5.050000,7.000000,2940.000000,6.661800e+05,12.060000,150.000000,0.600000,0.588207
max,3950.000000,3755.250000,72.900000,124.000000,131530.000000,6.235000e+07,267.290000,850.000000,3.400000,inf


In [30]:
df_call.dtypes

Symbols                object
Option_type            object
Expiry                 object
Strike_price          float64
Spot_price            float64
BidAsk                float64
Orders                  int64
Volume                float64
Premium               float64
Diff(%)               float64
Date                   object
Days_diff               int64
Time_to_Expiry        float64
Implied_Volatility    float64
dtype: object

https://medium.datadriveninvestor.com/black-scholes-and-option-greeks-in-python-6038f184801e
- **Delta** : Measures Impact of a Change in the Price of Underlying [https://www.investopedia.com/terms/d/delta.asp]
- **Gamma**: Measures the Rate of Change of Delta
- **Theta**: Measures Impact of a Change in Time Remaining
- **Vega**: Measures Impact of a Change in Volatility 
- **Rho**: Measures the impact of changes in Interest rates

In [31]:
import numpy as np
from scipy.stats import norm

def calculate_delta(row):
    S = row['Spot_price']  # Spot price
    K = row['Strike_price']  # Strike price
    r = 0  # Risk-free interest rate 
    T = row['Time_to_Expiry']  # Time to expiry in years
    sigma = row['Implied_Volatility']  # Implied volatility

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    delta = norm.cdf(d1)

    if row['Option_type'] == 'put':
        delta -= 1  # Adjust delta for put options

    return delta

# Apply the delta calculation function to each row in the DataFrame
df_call['Delta'] = df_call.apply(calculate_delta, axis=1)

df_call.head(5)

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff,Time_to_Expiry,Implied_Volatility,Delta
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126,0.504,2.082404,0.760318
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581,2.324,0.754821,0.405748
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7,0.028,0.641070,0.456486
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581,2.324,0.268521,0.565635
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581,2.324,0.294886,0.551220


In [32]:
hjfjhf

NameError: name 'hjfjhf' is not defined

In [34]:
import py_vollib
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks import analytical

# Assuming your dataframe is called df_call
for index, row in df_call.iterrows():
    S = row['Spot_price']  # Spot price
    K = row['Strike_price']  # Strike price
    r = 0  # Risk-free interest rate (assumed to be 0 for simplicity)
    t = row['Days_diff'] / 250 # Time to expiry in years
    sigma = row['Implied_Volatility']  # Implied volatility
    option_type = row['Option_type']  # Option type (either 'call' or 'put')

    # Calculate the Delta using the analytical method
    delta_new = analytical.delta('call', S, K, t, r, sigma)
    
    # Assign the calculated Delta value to the dataframe
    df_call.loc[index, 'Delta_new'] = delta_new


In [35]:
df_call.head(5)

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff,Time_to_Expiry,Implied_Volatility,Delta,Delta_new
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126,0.504,2.082404,0.760318,0.760318
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581,2.324,0.754821,0.405748,0.405748
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7,0.028,0.641070,0.456486,0.456486
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581,2.324,0.268521,0.565635,0.565635
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581,2.324,0.294886,0.551220,0.551220


In [36]:
import numpy as np
from scipy.stats import norm

# Constants
risk_free_rate = 0  # Example value, replace with your own
df_call['Time_to_Expiry'] = df_call['Time_to_Expiry']

# Calculate Vega
df_call['Vega'] = df_call['Spot_price'] * np.exp(-risk_free_rate * df_call['Time_to_Expiry']) * norm.pdf(df_call['Implied_Volatility']) * np.sqrt(df_call['Time_to_Expiry'])

In [37]:
df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff,Time_to_Expiry,Implied_Volatility,Delta,Delta_new,Vega
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126,0.504,2.082404,0.760318,0.760318,0.309055
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581,2.324,0.754821,0.405748,0.405748,10.758329
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7,0.028,0.641070,0.456486,0.456486,4.593630
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581,2.324,0.268521,0.565635,0.565635,5.772530
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581,2.324,0.294886,0.551220,0.551220,22.307845


In [43]:

df_call.head()

,Symbols,Option_type,Expiry,Strike_price,Spot_price,BidAsk,Orders,Volume,Premium,Diff(%),Date,Days_diff,Time_to_Expiry,Implied_Volatility,Delta,Vega
0,ISEE,c,2022-10-21,10.0,9.54,5.05,7,360.0,183600.0,4.71,2022-06-17,126,0.504,2.082404,0.760318,0.309055
1,CVNA,c,2024-01-19,60.0,23.52,4.60,7,634.0,310660.0,155.05,2022-06-17,581,2.324,0.754821,0.405748,10.758329
3,TWLO,c,2022-06-24,86.0,84.51,2.95,5,722.0,198800.0,2.48,2022-06-17,7,0.028,0.641070,0.456486,4.593630
8,ET,c,2024-01-19,10.0,9.84,1.53,6,1200.0,186160.0,1.75,2022-06-17,581,2.324,0.268521,0.565635,5.772530
9,WFC,c,2024-01-19,40.0,38.31,6.15,5,250.0,152250.0,4.98,2022-06-17,581,2.324,0.294886,0.551220,22.307845


To calculate Vega in Python for the given DataFrame df_call

Vega = Spot_price * e^(-r * T) * N'(d1) * sqrt(T)

Where:

Spot_price is the current price of the underlying asset.
r is the risk-free interest rate.
T is the time to expiry in years.
N'(d1) is the standard normal probability density function evaluated at d1.
d1 is the input of the cumulative distribution function for the standard normal distribution.